In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
import ast
import numpy as np

from utils import count_parameter_occurence, update_stability, load_model_variables, build_analyse_design_space

# Table 1: Phenotype and LC counts per model
## Complete

In [2]:
model_list = ['Design 1A','Design 1B','Design 1C','Design 2','Design 3',
              'Design 4','Design 5','Design 6','Design 7','Design 8','Design 9']

df = pd.DataFrame()
for i, model in enumerate(model_list):
    
    path = './Figures/'+model+'/model_properties.txt'
    with open(path, 'r') as file:
        model_props = file.read().replace('\n', '')
        model_props = ast.literal_eval(model_props)
      
    df.at[model, 'Total phenotypes'] = model_props['number of phenotypes']
    df.at[model, 'Valid phenotypes'] = model_props['number of valid phenotypes']
    
    path = './Figures/'+model+'/Stability.xlsx'
    df_stability = pd.read_excel(path)
    df.at[model, 'Phenotypes with oscillatory potential'] = len(df_stability[df_stability['Oscillatory potential']])
    
    path = './Figures/'+model+'/LC_parameters.xlsx'
    df_LC = pd.read_excel(path)
    df.at[model, 'Phenotypes with sampled LC'] = len(list(set(df_LC.index)))
    
    df.at[model, 'LCs retrieved'] = len(df_LC.index)
        
    df_LC = df_LC[df_LC['Peak sequence'].isin(['s, x, y, z','x, y, z, s', 'y, z, s, x', 'z, s, x, y'])]
    df.at[model, 'Phenotypes LC with correct peak sequence'] = len(list(set(df_LC.index)))
    df.at[model, 'LCs with correct peak sequence'] = len(df_LC.index)
    
    df_LC = df_LC[df_LC['Min. Amplitude (min/max)'] <= 0.5]
    df.at[model, 'LCs with correct peak sequence and 50% oscillation amplitude'] = len(df_LC.index)
    
df = df.astype(int)
df

,Total phenotypes,Valid phenotypes,Phenotypes with oscillatory potential,Phenotypes with sampled LC,LCs retrieved,Phenotypes LC with correct peak sequence,LCs with correct peak sequence,LCs with correct peak sequence and 50% oscillation amplitude
Design 1A,1105920,491,0,0,0,0,0,0
Design 1B,8847360,5969,62,10,19,0,0,0
Design 1C,2799360,4282,43,10,12,0,0,0
Design 2,4199040,3519,26,7,12,0,0,0
Design 3,995328,3335,664,18,26,7,8,3
Design 4,1990656,6689,1158,97,281,66,204,141
Design 5,1990656,6950,1523,32,53,16,23,13
Design 6,1990656,6844,1508,40,59,20,31,20
Design 7,1990656,6692,1239,46,60,19,24,15
Design 8,1990656,6750,974,30,31,9,9,7


## Reduced for publication

In [3]:
df = df.drop(['Phenotypes with sampled LC','LCs retrieved','LCs with correct peak sequence and 50% oscillation amplitude'],axis=1)
df = df.rename({'':''})
df

,Total phenotypes,Valid phenotypes,Phenotypes with oscillatory potential,Phenotypes LC with correct peak sequence,LCs with correct peak sequence
Design 1A,1105920,491,0,0,0
Design 1B,8847360,5969,62,0,0
Design 1C,2799360,4282,43,0,0
Design 2,4199040,3519,26,0,0
Design 3,995328,3335,664,7,8
Design 4,1990656,6689,1158,66,204
Design 5,1990656,6950,1523,16,23
Design 6,1990656,6844,1508,20,31
Design 7,1990656,6692,1239,19,24
Design 8,1990656,6750,974,9,9


# Table 2: Parameter occurence in dominant terms

## Phenotypes that show complex conjugate positive EVs
I.e. oscillatory potential

In [4]:
model_list = model_list[1:] # drop 1A

# Read all parameter occurence dataframes and join them
df = pd.DataFrame()
for i,model in enumerate(model_list):
    
    path = './Figures/'+model+'/Parameter_occurence_phenotypes_oscillatory_potential.xlsx'
    df_model = pd.read_excel(path)
    
    if i == 0:
        df = df_model.copy()
    else:
        df = df.append(df_model)
        
df.index = [m.capitalize().replace('_',' ').replace('Mondeel','Model') for m in model_list]
df.columns = [c.replace('_','') for c in df.columns]

# leave out v_x and v_s (always present)
df = df[['axy','ayy','axz','ayz','azz','gyx','gzx','gyy','gzy','gzz','d','e','l','bx','by','bz','bs','vy','vz']]
df = df.astype(int,errors='ignore')

df

,axy,ayy,axz,ayz,azz,gyx,gzx,gyy,gzy,gzz,d,e,l,bx,by,bz,bs,vy,vz
Design 1b,10,52,15,17,20,17,26,0,47,1,19,26,17.0,12,3,7,0,0,10
Design 1c,7,36,11,11,14,11,17,0,33,1,18,25,NaN,8,2,5,0,0,7
Design 2,4,22,7,7,8,6,9,0,21,1,0,26,NaN,4,1,3,0,0,4
Design 3,276,161,186,217,108,173,204,191,209,245,637,317,NaN,214,158,222,0,227,153
Design 4,448,314,282,388,230,298,342,319,363,357,1081,661,NaN,342,269,352,0,396,258
Design 5,626,365,436,523,220,417,491,436,494,558,1392,748,NaN,455,336,507,25,532,344
Design 6,634,354,421,500,222,424,470,439,472,548,1390,744,NaN,455,344,495,14,520,365
Design 7,530,322,349,400,200,339,390,356,408,459,1160,614,NaN,388,268,394,0,387,290
Design 8,340,323,253,293,191,278,310,257,354,342,867,536,NaN,267,190,294,0,311,237
Design 9,284,171,179,241,107,175,211,186,220,243,649,339,NaN,218,155,218,0,222,150


## In phenotypes that show LCs
- Design 1A does not have limit cycles
- Activatory parameters
    - Design 4: a_zz 
    - Design 1B-3, 5-9: a_yy the highest 
    - Design 3, 5-7, 9: a_yz as the second highest
- Inhibitory parameters
    - Design 1B-9: g_zy the highest

In [5]:
# Read all parameter occurence dataframes and join them
df = pd.DataFrame()
for i,model in enumerate(model_list):
    
    path = './Figures/'+model+'/Parameter_occurence_phenotypes_LC.xlsx'
    df_model = pd.read_excel(path)
    
    if i >= 4: # Design 4-9
        par_name = [x for x in df_model.columns if x != 'K_A' and x[:2] == 'K_'][0]
        df_model = df_model.rename({par_name:'K'},axis=1)
    else:
        df_model['K'] = np.nan
    
    if i == 0:
        df = df_model.copy()
    else:
        df = df.append(df_model)
        
df.index = [m.capitalize().replace('_',' ').replace('Mondeel','Model') for m in model_list]
df.columns = [c.replace('_','') for c in df.columns]

# leave out v_x and v_s (always present)
df = df[['axy','ayy','axz','ayz','azz','gyx','gzx','gyy','gzy','gzz','K','d','e','l','bx','by','bz','bs','vy','vz']]
df = df.astype(int,errors='ignore')

df

,axy,ayy,axz,ayz,azz,gyx,gzx,gyy,gzy,gzz,K,d,e,l,bx,by,bz,bs,vy,vz
Design 1b,3,7,4,2,4,1,4,0,7,0,NaN,3,5,2.0,3,0,0,0,0,0
Design 1c,1,9,8,0,1,2,2,0,7,1,NaN,4,6,NaN,2,0,1,0,0,1
Design 2,0,7,4,1,2,2,1,0,6,1,NaN,0,7,NaN,0,0,1,0,0,0
Design 3,0,15,2,12,4,4,9,2,11,4,NaN,17,11,NaN,5,1,4,0,3,0
Design 4,50,41,6,32,59,22,25,18,31,8,73.0,74,83,NaN,22,17,15,0,6,0
Design 5,3,27,8,19,5,11,11,1,22,1,17.0,25,25,NaN,9,1,10,2,2,0
Design 6,6,34,3,23,13,12,15,5,26,2,20.0,31,29,NaN,11,2,10,3,0,1
Design 7,8,35,8,23,8,14,18,6,29,3,28.0,42,28,NaN,13,3,18,0,3,7
Design 8,7,21,10,9,7,13,13,3,16,5,15.0,28,20,NaN,3,0,11,0,2,4
Design 9,2,22,3,17,4,6,11,2,19,1,6.0,22,14,NaN,5,1,9,0,0,0


## (Manuscript version) In phenotypes with LCs with correct peak sequence
- Design 1A-2: do not have limit cycles that satisfy these conditions
- Activatory parameters
    - Design 3: a_yy and a_yz are tied 
    - Design 4: a_zz 
    - Design 5-9: a_yy the highest and a_yz as the second highest
- Inhibitory parameters
    - Design 5: g_zy followed by g_zx
    - Design 3-9: g_zx followed by g_zy (in Design 8 they are equal)
- Other
    - In Design 3-5,7-9 either delta or epsilon (D4 only) is the (tied) highest parameter overall. Attesting to the importance of Sic1 degradation by way of the Clbs.

In [6]:
# Read all parameter occurence dataframes and join them
df = pd.DataFrame()
for i,model in enumerate(model_list):
    
    path = './Figures/'+model+'/LC_parameters'
    df_LC = pd.read_pickle(path)
    
    # select the subset with OK amplitude
    df_LC = df_LC[df_LC['Peak sequence'].isin(['s, x, y, z','x, y, z, s', 'y, z, s, x', 'z, s, x, y'])]
    
    # get the parameter counts for this subset
    LC_phenos = list(set(df_LC.index.tolist())) # only unique phenos
    
    scipy_model, pset, variables, y0, f, constraints, parbounds, latex_symbols, varnames = load_model_variables(model.replace('D','d').replace(' ','_'))
    ds, valid_cases = build_analyse_design_space(model, f, constraints, latex_symbols)
    df_model = count_parameter_occurence(ds, LC_phenos)
    
    if i >= 4: # Design 4-9
        par_name = [x for x in df_model.columns if x != 'K_A' and x[:2] == 'K_'][0]
        df_model = df_model.rename({par_name:'K'},axis=1)
    else:
        df_model['K'] = np.nan
    
    if i == 0:
        df = df_model.copy()
    else:
        df = df.append(df_model)
        
df.index = [m.capitalize().replace('_',' ').replace('Mondeel','Model') for m in model_list]
df.columns = [c.replace('_','') for c in df.columns]

# leave out v_x and v_s (always present)
df = df[['axy','ayy','axz','ayz','azz','gyx','gzx','gyy','gzy','gzz','K','d','e','l','bx','by','bz','bs','vy','vz']]
df = df.astype(int,errors='ignore')

df

,axy,ayy,axz,ayz,azz,gyx,gzx,gyy,gzy,gzz,K,d,e,l,bx,by,bz,bs,vy,vz
Design 1b,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0.0,0,0,0,0,0,0
Design 1c,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0,0,0,0,0,0
Design 2,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0,0,0,0,0,0
Design 3,0,6,0,6,1,1,5,1,4,0,NaN,7,3,NaN,1,0,4,0,1,0
Design 4,41,23,1,17,48,12,20,12,18,4,54.0,46,63,NaN,14,11,5,0,2,0
Design 5,3,12,2,10,4,0,11,0,13,0,8.0,14,12,NaN,5,1,4,0,1,0
Design 6,3,17,0,13,7,3,13,2,12,1,10.0,16,15,NaN,3,2,4,2,0,0
Design 7,6,12,2,9,7,3,14,3,12,0,16.0,18,13,NaN,2,1,6,0,1,1
Design 8,2,6,3,3,2,2,6,0,6,2,2.0,9,6,NaN,1,0,3,0,1,1
Design 9,2,11,1,9,3,2,11,1,10,1,1.0,12,7,NaN,0,1,5,0,0,0
